Ingest PRAM(Pregnancy Risk Assesment Montoring) Health Data CSV Source File
URL: https://materncaredatalakehouse.file.core.windows.net/pramfile/PregnancyRiskAssesment.csv



In [0]:
processName = dbutils.widgets.get('prm_processName')

nextSourceFileDateSql = f"""SELECT NVL(MAX(PROCESSED_FILE_TABLE_DATE)+1,'2025-06-01')  as NEXT_SOURCE_FILE_DATE FROM materncare_anlytics.processlogs.deltalakehouse_process_run 
WHERE PROCESS_NAME = '{processName}' and PROCESS_STATUS='Completed'"""


nextSourceFileDateDF = spark.sql(nextSourceFileDateSql)
display(nextSourceFileDateDF)

In [0]:

PRAMourceLayerName= "materncare-analytics"
PRAMSourceStorageName= "materncaredatalakehouse"
PRAMSourceFileName= "PregnancyRiskAssesment.csv"


PRAMSinkLayerName= "bronze"
PRAMSinkStorageName= "materncaredatalakehouse"
PRAMStorageFolderName="Pregnancy-Data"




In [0]:
%python
import pandas as pds

In [0]:
%python
%pip install fsspec

from pyspark.sql.functions import current_timestamp
PRAMSourceFolderPath = f"abfss://{PRAMourceLayerName}@{PRAMSourceStorageName}.dfs.core.windows.net/{PRAMSourceFileName}"
print(PRAMSourceFolderPath)

PRAMSparkDF = spark.read.format('csv').option('header', 'true').load(PRAMSourceFolderPath); PRAMPandaDF = PRAMSparkDF.toPandas()
print(PRAMPandaDF)

In [0]:
PRAMSparkDFWrite =  spark.createDataFrame(PRAMPandaDF)

In [0]:
from pyspark.sql.functions import current_timestamp
PRAMSinkFolderPath = f"abfss://{PRAMSinkLayerName}@{PRAMSinkStorageName}.dfs.core.windows.net/{PRAMStorageFolderName}"
print(PRAMSinkFolderPath)
# Remove or cast the unsupported column
PRAMSparkDFWrite = PRAMSparkDFWrite.drop("Data_Value_Std_Err")

(
    PRAMSparkDFWrite
    .withColumn("source_file_load_date",current_timestamp())
    .write
    .mode("append")
    .option("header","true")
    .csv(PRAMSinkFolderPath)

)

In [0]:
processFileDate = nextSourceFileDateDF.select('NEXT_SOURCE_FILE_DATE').collect()[0]['NEXT_SOURCE_FILE_DATE']
processStatus ='Completed'

processInsertSql = f""" INSERT INTO materncare_anlytics.processlogs.deltalakehouse_process_run(PROCESS_NAME,PROCESSED_FILE_TABLE_DATE,PROCESS_STATUS) VALUES('{processName}','{processFileDate}','{processStatus}')"""

spark.sql(processInsertSql)